<a href="https://colab.research.google.com/github/SaiKrishna022399/SaiKrishna_INFO5731_Fall2022/blob/main/Behara_Pavan_Kumar_In_class_exercise_04_03282023_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [3]:
# Write your code here
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import pandas as pd
Headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
}

titles = []
text=[]
for pg in range(1,4000):
  url = 'https://www.amazon.com/product-reviews/B08PNKRPCY/ref=acr_dp_hist_5?ie=UTF8&filterByStar=five_star&reviewerType=all_reviews'
  pg = requests.get(url, headers=Headers)
  s = BeautifulSoup(pg.content,'html.parser')
  reviews = s.find_all('a',class_='review-title-content')
  for i in range(len(reviews)):
    titles.append(reviews[i].get_text().strip())
  reviews= s.find_all('span',class_='review-text-content')
  for r in range(0,len(reviews)):
    text.append(reviews[r].get_text().strip()) 
reviews_df = pd.DataFrame({
    'Title' : titles,
    'Review': text
})

# Save the DataFrame as a CSV file
reviews_df.to_csv('reviews.csv', index=False)

# Read the saved CSV file and print the first few rows
data_1 = pd.read_csv('reviews.csv')
print(data_1.head())
print(f'Shape of the data frame: {reviews_df.shape}')
reviews_df

from google.colab import files

# the data is downloaded in to local machine
files.download('reviews.csv')

                                    Title  \
0                               Great Buy   
1                                Amazing!   
2      I love it! No scratches or dents..   
3  IPhone Mini is the perfect size for me   
4                  rlly good !! Recommend   

                                              Review  
0  I ordered my iPhone on 2/21/23, and have waite...  
1  I was hesitant at first but after recieving th...  
2  I was worried about ordering while reading the...  
3  I can use this phone with one hand. Top left c...  
4  The battery at 86 still good n everything work...  
Shape of the data frame: (1680, 2)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
data_1 = pd.read_csv('reviews.csv')
data_1.head()

,Title,Review
0,Great Buy,"I ordered my iPhone on 2/21/23, and have waite..."
1,Amazing!,I was hesitant at first but after recieving th...
2,I love it! No scratches or dents..,I was worried about ordering while reading the...
3,IPhone Mini is the perfect size for me,I can use this phone with one hand. Top left c...
4,rlly good !! Recommend,The battery at 86 still good n everything work...


In [5]:
import re
# Remove punctuation
data_1['Review_processed'] = \
data_1['Review'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
data_1['Review_processed'] = \
data_1['Review_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
data_1['Review_processed'].head()

0    i ordered my iphone on 2/21/23 and have waited...
1    i was hesitant at first but after recieving th...
2    i was worried about ordering while reading the...
3    i can use this phone with one hand top left co...
4    the battery at 86 still good n everything work...
Name: Review_processed, dtype: object

In [6]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = data_1.Review_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['ordered', 'iphone', 'waited', 'write', 'review', 'bc', 'wanted', 'test', 'first', 'afraid', 'order', 'refurbished', 'phone', 'glad', 'phone', 'arrived', 'day', 'promised', 'expected', 'iphone', 'scratches', 'battery', 'capacity', 'got', 'iphone', 'looks', 'brand', 'new', 'scratches', 'damage']


In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [8]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.044*"phone" + 0.029*"new" + 0.021*"battery" + 0.019*"days" + 0.018*"worried" + 0.018*"arrived" + 0.018*"bad" + 0.018*"reviews" + 0.017*"brand" + 0.016*"avoid"'), (1, '0.056*"phone" + 0.038*"amazing" + 0.028*"battery" + 0.026*"charge" + 0.025*"new" + 0.021*"iphone" + 0.020*"well" + 0.018*"arrived" + 0.015*"minutes" + 0.014*"recommend"'), (2, '0.071*"phone" + 0.022*"brand" + 0.021*"new" + 0.021*"came" + 0.019*"arrived" + 0.018*"battery" + 0.018*"looked" + 0.018*"first" + 0.012*"iphone" + 0.011*"ok"'), (3, '0.043*"phone" + 0.029*"battery" + 0.024*"one" + 0.021*"hand" + 0.020*"also" + 0.015*"smaller" + 0.015*"adjust" + 0.015*"left" + 0.015*"charger" + 0.013*"bad"'), (4, '0.034*"phone" + 0.031*"new" + 0.026*"brand" + 0.018*"xr" + 0.017*"bad" + 0.017*"days" + 0.017*"reviews" + 0.016*"case" + 0.015*"iphone" + 0.015*"battery"'), (5, '0.051*"phone" + 0.048*"battery" + 0.020*"recommend" + 0.017*"would" + 0.016*"arrived" + 0.016*"avoid" + 0.016*"credit" + 0.016*"proof" + 0.016*"definitely

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [9]:
# Write your code here
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(), 
                                remove_stopwords, 
                                strip_punctuation, 
                                strip_short, 
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

# apply function to all reviews 
data_1['Review_Text (Clean)'] = data_1['Review'].apply(lambda x: preprocess(x))
# preview of dataset
data_1.head()

,Title,Review,Review_processed,Review_Text (Clean)
0,Great Buy,"I ordered my iPhone on 2/21/23, and have waite...",i ordered my iphone on 2/21/23 and have waited...,"[order, iphon, wait, write, review, want, test..."
1,Amazing!,I was hesitant at first but after recieving th...,i was hesitant at first but after recieving th...,"[hesit, reciev, shock, want, upgrad, iphon, ma..."
2,I love it! No scratches or dents..,I was worried about ordering while reading the...,i was worried about ordering while reading the...,"[worri, order, read, review, satisfi, damag, a..."
3,IPhone Mini is the perfect size for me,I can use this phone with one hand. Top left c...,i can use this phone with one hand top left co...,"[us, phone, hand, left, corner, bit, stretch, ..."
4,rlly good !! Recommend,The battery at 86 still good n everything work...,the battery at 86 still good n everything work...,"[batteri, good, work, rlly, good, 100, recom, ..."


In [10]:
from gensim import corpora

# create a dictionary with the corpus
corpus = data_1['Review_Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# Coherence score in topic modeling to measure how interpretable the topics are to humans.
# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=data_1['Review_Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.7751971346905133
Coherence score with 3 clusters: 0.817241187430132
Coherence score with 4 clusters: 0.7059806647954758
Coherence score with 5 clusters: 0.7582487007796781
Coherence score with 6 clusters: 0.7812215233539086
Coherence score with 7 clusters: 0.7389205824627892
Coherence score with 8 clusters: 0.8016726822451709
Coherence score with 9 clusters: 0.6864426179358752
Coherence score with 10 clusters: 0.7583775262387926


In [11]:
# perform SVD on the bag of words with the LsiModel to extract 2 topics
lsi = LsiModel(bow, num_topics=2, id2word=dictionary)

# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=10):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.655*"phone" + 0.219*"new" + 0.204*"iphon" + 0.196*"brand" + 0.185*"look" + 0.181*"batteri" + 0.157*"review" + 0.124*"arriv" + 0.118*"came" + 0.108*"order".
Words in 1: 0.581*"iphon" + -0.238*"phone" + 0.185*"capac" + 0.164*"scratch" + 0.151*"case" + 0.135*"batteri" + 0.124*"order" + -0.111*"go" + 0.109*"glad" + 0.109*"got".


In [12]:
# find the scores given between the review and each topic
corpus_lsi = lsi[bow]


score1 = []
score2 = []
for doc in corpus_lsi:
    if len(doc) > 0 and len(doc[0]) > 1:
        score1.append(round(doc[0][1],2))
    if len(doc) > 1 and len(doc[1]) > 1:
        score2.append(round(doc[1][1],2))
# create data frame that shows scores assigned for both topics for each review
# Remove any reviews that were not processed by the LDA model
data_1 = data_1.iloc[:len(score1)]

# Create a DataFrame with the review text and topic scores
topic_df = pd.DataFrame()
topic_df['Text'] = data_1['Review']
topic_df['Topic 0 score'] = score1
topic_df['Topic 1 score'] = score2
topic_df['Topic']= topic_df[['Topic 0 score', 'Topic 1 score']].apply(lambda x: x.argmax(), axis=1)

topic_df.head()

,Text,Topic 0 score,Topic 1 score,Topic
0,"I ordered my iPhone on 2/21/23, and have waite...",7.40,7.63,1
1,I was hesitant at first but after recieving th...,3.09,0.46,0
2,I was worried about ordering while reading the...,7.17,-2.76,0
3,I can use this phone with one hand. Top left c...,3.65,0.42,0
4,The battery at 86 still good n everything work...,0.89,0.38,0


In [13]:
topic_df0 = topic_df[topic_df['Topic'] == 0]
topic_df1 = topic_df[topic_df['Topic']==1]
print('Sample text from topic 0:\n {}'.format(topic_df0.sample(1, random_state=2)['Text'].values))
print('Sample text from topic 0:\n {}'.format(topic_df1.sample(1, random_state=2)['Text'].values))

Sample text from topic 0:
 ['I loved this I needed a new phone and the battery life is amazing I’ll charge it at 10% and then 5 minutes go by and it’s up to 40% it’s amazing and the charge last forever and the touchscreen is amazing works so well ! Highly recommend']
Sample text from topic 0:
 ["I ordered my iPhone on 2/21/23, and have waited to write a review bc I wanted to test it first.  I was afraid to order a refurbished phone, but I'm glad I did.  The phone arrived on the day it was promised.  I expected an iPhone with some scratches and battery capacity between 80 and 95%.  What I got was an iPhone that looks brand new with no scratches or damage of any kind.  Battery capacity is 100%.  Needless to say, I was pleasantly surprised.  It is in excellent condition.  Although the phone is small, it fits well in my hands.  I put a Diaclara designed full body case on my iPhone.  It is clear and the back has glitter on it.  The case makes the iPhone look a wee bit larger.  So far the iP

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [14]:
# Write your code here
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 29.5 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.2.0 which is incompatible.


In [15]:
import nltk
nltk.download('all')
!pip install preprocess
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import pyLDAvis
pyLDAvis.enable_notebook()
top = 10
topic_to_topwords = {}
for j, topic in enumerate(data_1['Review_Text (Clean)'].tolist()):
    top1 = np.argsort(topic)[::-1][:top]
    msg = 'Topic %i '  % j
    tp_wrd = [s.strip()[:35] for s in data_1['Review_Text (Clean)'][i] for i in top1]
    msg += ' has top words '.join(tp_wrd)
    print(msg)
    topic_to_topwords[j] = tp_wrd

Topic 0 order has top words order has top words order has top words order has top words order has top words order has top words order has top words order has top words order has top words order has top words iphon has top words iphon has top words iphon has top words iphon has top words iphon has top words iphon has top words iphon has top words iphon has top words iphon has top words iphon has top words wait has top words wait has top words wait has top words wait has top words wait has top words wait has top words wait has top words wait has top words wait has top words wait has top words write has top words write has top words write has top words write has top words write has top words write has top words write has top words write has top words write has top words write has top words review has top words review has top words review has top words review has top words review has top words review has top words review has top words review has top words review has top words review has to

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [17]:
# Write your code here
# Write your code here
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


data_1['clean_text'] = data_1.Review.apply(clean_text)
data_1.head()

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:21: DeprecationWarning: invalid escape sequence \(
<>:21: DeprecationWarning: invalid escape sequence \(
<ipython-input-17-4a7ea5f067c3>:21: DeprecationWarning: invalid escape sequence \(
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Title,Review,Review_processed,Review_Text (Clean),clean_text
0,Great Buy,"I ordered my iPhone on 2/21/23, and have waite...",i ordered my iphone on 2/21/23 and have waited...,"[order, iphon, wait, write, review, want, test...",ordered iphone waited write review bc wanted t...
1,Amazing!,I was hesitant at first but after recieving th...,i was hesitant at first but after recieving th...,"[hesit, reciev, shock, want, upgrad, iphon, ma...",hesitant first recieving shocked wanted upgrad...
2,I love it! No scratches or dents..,I was worried about ordering while reading the...,i was worried about ordering while reading the...,"[worri, order, read, review, satisfi, damag, a...",worried ordering reading reviews im satisfied ...
3,IPhone Mini is the perfect size for me,I can use this phone with one hand. Top left c...,i can use this phone with one hand top left co...,"[us, phone, hand, left, corner, bit, stretch, ...",use phone one hand top left corner bit stretch...
4,rlly good !! Recommend,The battery at 86 still good n everything work...,the battery at 86 still good n everything work...,"[batteri, good, work, rlly, good, 100, recom, ...",battery still good n everything works rlly goo...


In [18]:
!pip install --upgrade joblib

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
! pip install bertopic

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached bertopic-0.14.1-py2.py3-none-any.whl (120 kB)
  Using cached hdbscan-0.8.29.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 

In [ ]:
from bertopic import BERTopic

reviews= data_1.clean_text.to_list()
timestamp = data_1.Review.to_list()

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(reviews)

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

In [21]:
# Write your answer here (no code needed for this question)
"""
Topic Modelling is a widely-used technique to identify themes or topics within a collection of documents. One way to evaluate the effectiveness of a topic model is through the concept of Topic Coherence. There are several techniques available for topic modeling, including Latent Semantic Analysis (LSA) and Latent Dirichlet Allocation (LDA), as well as more recent approaches such as lda2vec and BERTopic.

In LDA, we use a dictionary and a corpus as inputs to the model, along with the number of topics we want to identify. We can use a visualization tool such as pyLDAvis to compare the coherence scores for different numbers of topics, and choose the optimal number of topics based on the coherence score. Generally, we want to select the number of topics that provides the highest coherence score while still producing meaningful and interpretable topics.

LSA is often used as a technique to reduce dimensionality or noise in data. We can also use Topic Coherence to determine the optimal number of topics for an LSA model. The highest coherence score indicates the best topic model. Comparing the coherence score for LDA and LSA, we can see that LDA generally provides better results.

In summary, both LDA and LSA can be used for topic modeling, but LDA tends to produce better results in terms of Topic Coherence. We can evaluate the effectiveness of a topic model by comparing the coherence scores for different numbers of topics, and selecting the model with the highest coherence score that still produces meaningful topics.
"""

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'\nTopic Modelling is a widely-used technique to identify themes or topics within a collection of documents. One way to evaluate the effectiveness of a topic model is through the concept of Topic Coherence. There are several techniques available for topic modeling, including Latent Semantic Analysis (LSA) and Latent Dirichlet Allocation (LDA), as well as more recent approaches such as lda2vec and BERTopic.\n\nIn LDA, we use a dictionary and a corpus as inputs to the model, along with the number of topics we want to identify. We can use a visualization tool such as pyLDAvis to compare the coherence scores for different numbers of topics, and choose the optimal number of topics based on the coherence score. Generally, we want to select the number of topics that provides the highest coherence score while still producing meaningful and interpretable topics.\n\nLSA is often used as a technique to reduce dimensionality or noise in data. We can also use Topic Coherence to determine the optima